IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\NO_FAULT2


In [3]:
guasto = 0

VOLO M2 NO FAULT

In [4]:
rcou_m2_nofault = pd.read_csv("RCOU.csv")
rcou_m2_nofault = rcou_m2_nofault.astype("float64")
rcou_m2_nofault = rcou_m2_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m2_nofault = rcou_m2_nofault[((rcou_m2_nofault['C9'] >= potenza) & (rcou_m2_nofault['C10'] >= potenza) & (rcou_m2_nofault['C11'] >= potenza) & (rcou_m2_nofault['C12'] >= potenza) & (rcou_m2_nofault['C13'] >= potenza) & (rcou_m2_nofault['C14']>= potenza))]

rcou_m2_nofault = rcou_m2_nofault.reset_index(drop=True)

display(rcou_m2_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,163535671.0,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0
1,163546021.0,1382.0,1426.0,1447.0,1359.0,1421.0,1387.0
2,163556077.0,1378.0,1442.0,1450.0,1368.0,1414.0,1407.0
3,163566849.0,1428.0,1405.0,1402.0,1431.0,1408.0,1425.0
4,163577867.0,1452.0,1391.0,1393.0,1450.0,1419.0,1426.0
...,...,...,...,...,...,...,...
7059,244588431.0,1556.0,1418.0,1490.0,1492.0,1444.0,1535.0
7060,244598703.0,1559.0,1408.0,1492.0,1484.0,1453.0,1521.0
7061,244609809.0,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0
7062,244620798.0,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0


In [5]:
min=rcou_m2_nofault['TimeUS'][0]
max=rcou_m2_nofault['TimeUS'][len(rcou_m2_nofault)-1]
print(max)
print(min)

244631223.0
163535671.0


In [6]:
xkf1_m2_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m2_nofault = xkf1_m2_nofault.astype("float64")
xkf1_m2_nofault = xkf1_m2_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_nofault = xkf1_m2_nofault[((xkf1_m2_nofault['TimeUS'] >= min) & (xkf1_m2_nofault['TimeUS'] <= max))]
xkf1_m2_nofault = xkf1_m2_nofault.reset_index(drop=True)
print(xkf1_m2_nofault)

           TimeUS  Roll  Pitch     Yaw
0     163543665.0  1.17  -0.58   13.28
1     163554261.0  1.14  -0.58   13.28
2     163564285.0  1.19  -0.56   13.29
3     163575443.0  1.23  -0.54   13.30
4     163586483.0  1.30  -0.52   13.28
...           ...   ...    ...     ...
7385  244586271.0 -4.88  -0.88  358.90
7386  244596618.0 -4.80  -0.96  358.86
7387  244607605.0 -4.77  -1.02  358.82
7388  244618752.0 -4.71  -1.02  358.80
7389  244629015.0 -4.67  -1.04  358.77

[7390 rows x 4 columns]


In [7]:
att_m2_nofault = pd.read_csv("ATT.csv")
att_m2_nofault = att_m2_nofault.astype("float64")
att_m2_nofault = att_m2_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_nofault = att_m2_nofault[((att_m2_nofault['TimeUS'] >= min) & (att_m2_nofault['TimeUS'] <= max))]
att_m2_nofault=att_m2_nofault.reset_index(drop=True)
print(att_m2_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0     163545292.0    -0.13  1.17     -0.07  -0.58   13.14   13.28
1     163555319.0    -0.15  1.14     -0.06  -0.58   13.14   13.28
2     163566291.0    -0.16  1.19     -0.05  -0.56   13.14   13.29
3     163577191.0    -0.18  1.23     -0.05  -0.54   13.14   13.30
4     163588078.0    -0.19  1.30     -0.04  -0.52   13.14   13.28
...           ...      ...   ...       ...    ...     ...     ...
7386  244587670.0    -2.61 -4.88     -0.90  -0.88    0.26  358.91
7387  244597835.0    -2.50 -4.80     -0.89  -0.96    0.26  358.87
7388  244609128.0    -2.42 -4.77     -0.87  -1.02    0.26  358.83
7389  244620166.0    -2.34 -4.71     -0.86  -1.02    0.26  358.81
7390  244630400.0    -2.23 -4.67     -0.85  -1.04    0.26  358.78

[7391 rows x 7 columns]


In [8]:
if len(att_m2_nofault)>len(xkf1_m2_nofault):
    to_add=att_m2_nofault[len(xkf1_m2_nofault):]
    att_m2_nofault=att_m2_nofault[:len(xkf1_m2_nofault)]

if len(xkf1_m2_nofault)>len(att_m2_nofault):
    to_add=xkf1_m2_nofault[len(att_m2_nofault):]

for idx,i in enumerate(att_m2_nofault['Roll']):
    if(xkf1_m2_nofault['Roll'][idx] != i):
        att_m2_nofault['Roll'][idx] = ((att_m2_nofault['Roll'][idx] + xkf1_m2_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m2_nofault['Pitch']):
    if(xkf1_m2_nofault['Pitch'][idx] != i):
        att_m2_nofault['Pitch'][idx] = ((att_m2_nofault['Pitch'][idx] + xkf1_m2_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_nofault['Yaw']):
    if(xkf1_m2_nofault['Yaw'][idx] != i):
        att_m2_nofault['Yaw'][idx] = ((att_m2_nofault['Yaw'][idx] + xkf1_m2_nofault['Yaw'][idx])/2)

att_m2_nofault=pd.concat([att_m2_nofault,to_add])

print(att_m2_nofault)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0     163545292.0    -0.13  1.170     -0.07  -0.58   13.14   13.280
1     163555319.0    -0.15  1.140     -0.06  -0.58   13.14   13.280
2     163566291.0    -0.16  1.190     -0.05  -0.56   13.14   13.290
3     163577191.0    -0.18  1.230     -0.05  -0.54   13.14   13.300
4     163588078.0    -0.19  1.300     -0.04  -0.52   13.14   13.280
...           ...      ...    ...       ...    ...     ...      ...
7386  244587670.0    -2.61 -4.840     -0.90  -0.92    0.26  358.885
7387  244597835.0    -2.50 -4.785     -0.89  -0.99    0.26  358.845
7388  244609128.0    -2.42 -4.740     -0.87  -1.02    0.26  358.815
7389  244620166.0    -2.34 -4.690     -0.86  -1.03    0.26  358.790
7390  244630400.0    -2.23 -4.670     -0.85  -1.04    0.26  358.780

[7391 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_nofault = pd.read_csv("ESC_0.csv")
esc_0_m2_nofault = esc_0_m2_nofault.astype("float64")
esc_0_m2_nofault = esc_0_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_nofault = esc_0_m2_nofault[((esc_0_m2_nofault['TimeUS'] >= min) & (esc_0_m2_nofault['TimeUS'] <= max))]
esc_0_m2_nofault=esc_0_m2_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_nofault = pd.read_csv("ESC_1.csv")
esc_1_m2_nofault = esc_1_m2_nofault.astype("float64")
esc_1_m2_nofault = esc_1_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_nofault = esc_1_m2_nofault[((esc_1_m2_nofault['TimeUS'] >= min) & (esc_1_m2_nofault['TimeUS'] <= max))]
esc_1_m2_nofault=esc_1_m2_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_nofault = pd.read_csv("ESC_2.csv")
esc_2_m2_nofault = esc_2_m2_nofault.astype("float64")
esc_2_m2_nofault = esc_2_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_nofault = esc_2_m2_nofault[((esc_2_m2_nofault['TimeUS'] >= min) & (esc_2_m2_nofault['TimeUS'] <= max))]
esc_2_m2_nofault=esc_2_m2_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_nofault = pd.read_csv("ESC_3.csv")
esc_3_m2_nofault = esc_3_m2_nofault.astype("float64")
esc_3_m2_nofault = esc_3_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_nofault = esc_3_m2_nofault[((esc_3_m2_nofault['TimeUS'] >= min) & (esc_3_m2_nofault['TimeUS'] <= max))]
esc_3_m2_nofault=esc_3_m2_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_nofault = pd.read_csv("ESC_4.csv")
esc_4_m2_nofault = esc_4_m2_nofault.astype("float64")
esc_4_m2_nofault = esc_4_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_nofault = esc_4_m2_nofault[((esc_4_m2_nofault['TimeUS'] >= min) & (esc_4_m2_nofault['TimeUS'] <= max))]
esc_4_m2_nofault=esc_4_m2_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_nofault = pd.read_csv("ESC_5.csv")
esc_5_m2_nofault = esc_5_m2_nofault.astype("float64")
esc_5_m2_nofault = esc_5_m2_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_nofault = esc_5_m2_nofault[((esc_5_m2_nofault['TimeUS'] >= min) & (esc_5_m2_nofault['TimeUS'] <= max))]
esc_5_m2_nofault=esc_5_m2_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m2_nofault = pd.read_csv("ESC_0.csv")\nesc_0_m2_nofault = esc_0_m2_nofault.astype("float64")\nesc_0_m2_nofault = esc_0_m2_nofault.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m2_nofault = esc_0_m2_nofault[((esc_0_m2_nofault[\'TimeUS\'] >= min) & (esc_0_m2_nofault[\'TimeUS\'] <= max))]\nesc_0_m2_nofault=esc_0_m2_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m2_nofault = pd.read_csv("ESC_1.csv")\nesc_1_m2_nofault = esc_1_m2_nofault.astype("float64")\nesc_1_m2_nofault = esc_1_m2_nofault.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m2_nofault = esc_1_m2_nofault[((esc_1_m2_nofault[\'TimeUS\'] >= min) & (esc_1_m2_nofault[\'TimeUS\'] <= max))]\nesc_1_m2_nofault=esc_1_m2_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\n

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m2_nofault = pd.read_csv("IMU_0.csv")
imu_0_m2_nofault = imu_0_m2_nofault.astype("float64")
imu_0_m2_nofault = imu_0_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_nofault = imu_0_m2_nofault[((imu_0_m2_nofault['TimeUS'] >= min) & (imu_0_m2_nofault['TimeUS'] <= max))]

imu_1_m2_nofault = pd.read_csv("IMU_1.csv")
imu_1_m2_nofault = imu_1_m2_nofault.astype("float64")
imu_1_m2_nofault = imu_1_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_nofault = imu_1_m2_nofault[((imu_1_m2_nofault['TimeUS'] >= min) & (imu_1_m2_nofault['TimeUS'] <= max))]

imu_2_m2_nofault = pd.read_csv("IMU_2.csv")
imu_2_m2_nofault = imu_2_m2_nofault.astype("float64")
imu_2_m2_nofault = imu_2_m2_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_nofault = imu_2_m2_nofault[((imu_2_m2_nofault['TimeUS'] >= min) & (imu_2_m2_nofault['TimeUS'] <= max))]

imu_m2_nofault = pd.concat((imu_0_m2_nofault, imu_1_m2_nofault, imu_2_m2_nofault))
imu_m2_nofault=imu_m2_nofault.groupby(imu_m2_nofault.TimeUS, as_index=False).mean()

display(imu_m2_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,163537245.0,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200
1,163539870.0,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074
2,163542660.0,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492
3,163545177.0,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348
4,163547584.0,0.044315,-0.006546,0.023296,-0.371309,0.336459,-9.679198
...,...,...,...,...,...,...,...
28162,244619831.0,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874
28163,244622275.0,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187
28164,244624838.0,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224
28165,244627437.0,0.037260,0.028643,-0.041810,-0.307790,0.681670,-9.576567


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m2_nofault = pd.merge_ordered(imu_m2_nofault,att_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m2_nofault = pd.merge_ordered(m2_nofault, esc_0_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_1_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_2_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_3_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_4_m2_nofault)
m2_nofault = pd.merge_ordered(m2_nofault, esc_5_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")'''
m2_nofault = pd.merge_ordered(m2_nofault, rcou_m2_nofault)
m2_nofault=m2_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m2_nofault["TimeUS"] = m2_nofault["TimeUS"] - m2_nofault.iloc[0]["TimeUS"]
m2_nofault["TimeUS"] = m2_nofault["TimeUS"].astype(int)
m2_nofault["Guasto"] = guasto

In [13]:
last_cell = m2_nofault.iloc[-1, m2_nofault.columns.get_loc('TimeUS')]
print(last_cell)

81095552


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
28380  81081660
28381  81084517
28382  81087374
28383  81090231
28384  81093088

[28385 rows x 1 columns]


In [16]:
m2_nofault_final = pd.merge_ordered(m2_nofault,df)
m2_nofault_final=m2_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m2_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY      AccZ  \
0             0  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
1          1574  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
2          2857  0.073797  0.052690  0.029002 -0.099784  0.158494 -9.828200   
3          4199  0.125925  0.025622  0.027003 -0.082384  0.063078 -9.522074   
4          5714  0.125925  0.025622  0.027003 -0.082384  0.063078 -9.522074   
...         ...       ...       ...       ...       ...       ...       ...   
70985  81091766  0.037260  0.028643 -0.041810 -0.307790  0.681670 -9.576567   
70986  81093088  0.037260  0.028643 -0.041810 -0.307790  0.681670 -9.576567   
70987  81094562  0.107283 -0.079716 -0.056382 -0.343552  0.632744 -9.387979   
70988  81094729  0.107283 -0.079716 -0.056382 -0.343552  0.632744 -9.387979   
70989  81095552  0.107283 -0.079716 -0.056382 -0.343552  0.632744 -9.387979   

       DesRoll  Roll  DesPitch  Pitch  DesYaw     Y

In [17]:
m2_nofault_final = m2_nofault_final[m2_nofault_final.TimeUS.isin(to_be)]

In [18]:
m2_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
2,2857,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
4,5714,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
6,8571,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
10,11428,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348,-0.13,1.17,-0.07,-0.58,13.14,13.280,1382.0,1426.0,1447.0,1359.0,1421.0,1387.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70975,81081660,-0.006303,0.000132,-0.044976,-0.432384,0.569364,-9.512742,-2.42,-4.74,-0.87,-1.02,0.26,358.815,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
70979,81084517,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874,-2.34,-4.69,-0.86,-1.03,0.26,358.790,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
70982,81087374,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187,-2.34,-4.69,-0.86,-1.03,0.26,358.790,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0
70984,81090231,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224,-2.34,-4.69,-0.86,-1.03,0.26,358.790,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0


In [19]:
from datetime import timedelta
m2_nofault_final=m2_nofault_final.reset_index(drop=True)
m2_nofault_final['TimeUS'] = pd.to_datetime(m2_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
1,00:00:00.002857,0.073797,0.052690,0.029002,-0.099784,0.158494,-9.828200,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
2,00:00:00.005714,0.125925,0.025622,0.027003,-0.082384,0.063078,-9.522074,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
3,00:00:00.008571,0.030989,-0.038425,0.015617,-0.124611,-0.010491,-9.361492,-0.13,1.17,-0.07,-0.58,13.14,13.280,1390.0,1407.0,1412.0,1385.0,1400.0,1398.0,0.0
4,00:00:00.011428,-0.055869,-0.065410,0.012424,-0.162603,0.150488,-9.439348,-0.13,1.17,-0.07,-0.58,13.14,13.280,1382.0,1426.0,1447.0,1359.0,1421.0,1387.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28380,00:01:21.081660,-0.006303,0.000132,-0.044976,-0.432384,0.569364,-9.512742,-2.42,-4.74,-0.87,-1.02,0.26,358.815,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
28381,00:01:21.084517,0.128640,-0.060799,-0.040252,-0.261952,0.785947,-9.640874,-2.34,-4.69,-0.86,-1.03,0.26,358.790,1577.0,1376.0,1489.0,1481.0,1476.0,1493.0,0.0
28382,00:01:21.087374,0.190746,-0.016192,-0.038782,-0.462853,0.624572,-9.764187,-2.34,-4.69,-0.86,-1.03,0.26,358.790,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0
28383,00:01:21.090231,0.057046,0.056984,-0.032290,-0.438465,0.580401,-9.704224,-2.34,-4.69,-0.86,-1.03,0.26,358.790,1575.0,1371.0,1518.0,1443.0,1508.0,1454.0,0.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_nofault_final)/350)):
        V1=m2_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28576\381666698.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_28576\381666698.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.141540,0.034814,-0.873153,0.233984,104.0,98.0,11.235387,6.653416,0.014573,0.004017,-0.294723,0.064945,71.0,96.0,2.526027,1.354690,0.140811,0.008906,-1.413448,0.169434,67.0,66.0,2.234423,1.068886,-0.357343,0.044805,0.056658,0.415176,98.0,101.0,19.936756,12.277275,0.393492,0.210230,-0.384469,0.603711,106.0,103.0,31.474309,19.501524,-10.367847,0.571760,-0.244294,10.395306,0.0,107.0,146.521509,23.676653,-0.539003,8.307059,-0.353129,2.928125,97.0,96.0,14.797756,7.083587,-0.159601,0.108934,-1.382473,0.366192,98.0,96.0,18.320636,2.543409,16.099373,7.713696,-1.309274,16.336509,120.0,87.0,2.275067,-1.619548,1571.393162,3611.324982,1.416627,1572.538555,117.0,119.0,1259.177483,1132.409621,1462.233618,1768.225267,-0.503461,1462.836403,119.0,114.0,2277.119638,947.119880,1523.997151,2348.317135,0.502727,1524.765209,122.0,120.0,929.849420,675.722302,1515.148148,2006.315132,1.862846,1515.808204,117.0,118.0,795.749937,677.543327,1490.535613,2439.455157,-0.779135,1491.351373,116.0,113.0,1887.179827,1043.747982,1545.920228,3838.479333,0.372256,1547.157683,114.0,112.0,1598.493574,743.667630,0
1,1.0,-0.141375,0.034725,-0.867272,0.233694,99.0,93.0,11.157941,6.521929,0.014483,0.004008,-0.291985,0.064859,71.0,91.0,1.680502,1.200857,0.140790,0.008880,-1.409273,0.169343,71.0,70.0,2.240340,1.068701,-0.357137,0.044692,0.064362,0.414864,100.0,103.0,20.535814,12.462846,0.395265,0.210737,-0.402594,0.605288,118.0,115.0,22.412463,20.007607,-10.366000,0.571332,-0.236784,10.393444,0.0,107.0,146.319578,23.791968,-0.557585,8.404940,-0.374899,2.948214,99.0,98.0,15.999453,7.224692,-0.160312,0.108802,-1.379581,0.366323,104.0,102.0,18.398776,2.482795,16.114006,7.767086,-1.313918,16.352559,117.0,116.0,2.252984,-2.421079,1571.553977,3610.139528,1.423939,1572.698886,116.0,118.0,1310.363712,1180.752248,1462.130682,1766.917347,-0.504581,1462.733068,119.0,114.0,2268.506658,1076.328265,1524.062500,2343.129986,0.515011,1524.828835,112.0,110.0,870.080431,583.157645,1515.170455,2000.774281,1.877482,1515.828683,108.0,109.0,799.780970,590.040962,1490.366477,2442.574713,-0.788017,1491.183380,120.0,117.0,1882.891108,972.379719,1546.144886,3845.309432,0.370647,1547.384372,113.0,111.0,1612.572472,818.437971,0
2,2.0,-0.141245,0.034632,-0.861153,0.233418,108.0,103.0,11.054809,6.191746,0.014298,0.004009,-0.300103,0.064824,85.0,69.0,1.206865,1.140452,0.140756,0.008856,-1.405439,0.169241,61.0,60.0,2.242517,1.063994,-0.356894,0.044586,0.071463,0.414527,96.0,99.0,19.994133,12.606075,0.396530,0.210703,-0.412121,0.606088,115.0,123.0,19.588048,15.213889,-10.364015,0.571099,-0.229855,10.391453,0.0,111.0,146.161024,23.943963,-0.576062,8.501577,-0.396842,2.968053,93.0,92.0,17.206269,7.373516,-0.161020,0.108669,-1.376646,0.366455,96.0,94.0,18.463930,2.396217,16.12

In [23]:
path_file = path_file.replace(r"M2\NO_FAULT2", "")
os.chdir(path_file)
df_merged.to_csv('m2_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
